### ***1.电商定向广告和搜索广告有怎样的区别，算法模型是否有差别？ ***

#### ***电商定向广告和搜索广告的差别可以从业务和算法两个不同角度来考虑 ***

| 业务        |定向广告    |搜索广告     |
| :--------: | :-------- | :-------- |
|场景        |用户浏览某一页面     |用户搜索具体关键字          |
|用户        |用户没有很明显的意图  |用户主动Query查询          | 
|推荐依据     |用户的历史行为和上下文信息      |用户的历史行为，上下文信息和当前搜索关键词|
|任务要求     |既要符合用户的兴趣同时也要兼顾全局热度和随机性 |兼顾用户的兴趣和与当前搜索关键字的匹配程度|


|算法         |定向广告    |搜索广告     |
|:-----------:|:---------|:----------|
|训练数据      |一系列特征数据与单个目标广告的信息对| 一系列特征数据与目标广告的信息序列，包含Pairwise的排序信息|
|算法模型      |排序算法,如：RankNet, LambdaRank 和 LambdaMART|经典的CTR预估模型，如：LR, MLR, DIN等 |
|评价指标      |交叉熵损失，AUC，Accuracy等|MAP, NDCG，MRR|



### ***2. 定向广告都有哪些常见的使用模型，包括Attention机制模型？***

定向广告的预测模型大致可分为线性模型，非线性模型和深度学习三种类型。
- LR 线性模型 
    - LR 模型 + 人工特征，LR 模型不能处理非线性特征，所以需要特征工程去加入非线性特征。
- 非线性模型
    - Mixed Logistic Regression, 混合逻辑回归。 
    - 树模型 1) 基础树模型 ID3, CART 等。 2) 集成模型：Bagging - RandomForest, Boosting - AdaBoost, GBDT.
- 深度学习模型
    - 基本的DNN模型，Embedding + MLP
    - DIN在目标广告与历史行为数据之间引入Attention Unit.
    - 更进一步有DIEN，DSIN 等

### ***3.DIN中的Attention机制思想和原理是怎样的？ ***

Attention机制的算法动机是，不是所有用户历史交互过的商品对当前广告的推荐计算都有相同的影响权重。直观上考虑，那些与目标广告具有属性相似度高的物品应该获得更大的影响权重。比如当前广告是关于电脑配件的，那么相比于用户曾经购买过的UGG雪地靴，蓝牙音箱应当得到的影响权重更大。

具体来讲：
- 目标广告信息与历史交互过的每一个商品信息并列通过一个可学习的Attention Unit来学习当前商品的在下一层Sum Pooling 计算的权重。
- 在Attention Unit内部，目标广告信息与商品信息做外积，然后再与两类信息合并，进入全连接层，计算权重值。
- 由于每个历史商品都有一个Attention Unit,数量很大；为了降低计算耗时，Attention Unit的内部结构不易过于复杂。

### ***4. DIEN相比于DIN有哪些创新？***

相比于DIN，DIEN结构设计的动机是：
1. 用户的行为和兴趣是序列化的，用户当前的行为可能受到前一时刻兴趣的影响。而DIN中针对于每个用户行为进行独立的兴趣抽取可能会存在问题。
2. 用户的兴趣会随着季节，身份，家庭环境，个人心态等因素的变化而发生变化；也就是说用户的兴趣随着时间推移会是不断进化的。而在DIN中没有办法表现这种兴趣的进化。

针对于以上以上问题，DIEN做出了一下改进：
1. 在Embedding layer 和 Concatenate Layer 之间加入按照序列提取兴趣的 Interest Extractor Layer 和 模拟兴趣演化的 Interest Evolving Layer. 
2. Interest Extractor Layer 使用了GRU的结构抽取了每一个时间片内用户的兴趣。
3. Interest Evolving Layer 利用序列模型AUGRU的结构将不同时间的用户兴趣串联起来，形成兴趣进化的链条。
4. 引入序列模型AUGRU模拟了用户兴趣进化的过程。

此外
1. 在兴趣抽取层，引入了auxiliary loss，以及下一时刻的真实行为和负样本进行额外的监督学习，促使GRU在提炼兴趣表达上更高效。
2. 采用GRU而非LSTM, 在算法性能相差不大的前提下，GRU模型的参数更少，训练速度更快，更加适合电子商务场景。

![](images/DIEN.png)

***Figure1*** - DIEN - 深度兴趣进化网络

### ***5. DSIN关于Session的洞察是怎样的，如何对Session兴趣进行表达？ ***

**DSIN 的设计动机**
1. 从大时间尺度来看，用户在一段时间内，比如3~6个月的时间；由于身份，关系状态或者所处的环境发生变化，兴趣也会随之变化；但是在小时间尺度上来看，用户会对某一类商品会有兴趣，即在某一段时间内浏览的商品之间有很强的相关性，而过一段时间这个小尺度的兴趣又会发生变化。
2. 从经验来看，这个小尺度的时间段通常是30分钟，将之定义为一段会话（Session）。
3. 会话内用户与商品交互的历史交互信息满足短时间的兴趣规律，同时在由会话组成的长时间的历史序列又满足DIEN兴趣演进的规律。

**DSIN 设计与Session兴趣表达**
1. 在对用户历史交互信息序列的处理，和兴趣抽取上。DSIN网络分别设计了，序列切分层（Session Division Layer），会话兴趣抽取层（Session Interest Extractor Layer）, 会话间兴趣交互层（Session Interest Interacting Layer）以及 会话兴趣激活层（Session Interest Activating Layer）。
2. 在序列切分层，按照经验参数将序列交互行为切分到一系列的会话中。
3. 在会话兴趣抽取层，通过自注意力机制（Self-attention）学习会话内部交互行为之间的关系。
4. 在会话间兴趣抽取层，将从会话兴趣抽取层抽取出来的会话兴趣序列通过一个双向长短记忆网络（Bi-LSTM）来抽取一个会话间的兴趣序列。
5. 在会话兴趣激活层，将要评估的物品与会话兴趣序列和会话间兴趣序列分别通过一个激活单元，来分别计算这两个兴趣序列中每个兴趣在当前序列中应该赋予的权重。并将两个序列的兴趣加权求和计算得到两个兴趣向量。然后将这两个兴趣向量拼接，传入下一层。

![](images/DSIN.png)

***Figure2*** - DSIN - 深度会话兴趣网络

### ***6. 如果你来设计淘宝定向广告，会有哪些future work（即下一个阶段的idea）？***